In [29]:
# Dependencies
%matplotlib osx
import stf
import eventanalysis
import utilities
import matplotlib.pyplot as plt
import minievents
import numpy as np
from scipy import stats, interpolate, optimize, integrate
import json

In [35]:
# Reload when edits happen
reload(utilities)
reload(stf)
reload(eventanalysis)

Reading in file


<module 'eventanalysis' from 'eventanalysis.pyc'>

In [32]:
# Analyse big events and remove noise
reload(eventanalysis)
EA = eventanalysis.EventAnalysis()
EA.run()

Identified events
Merged events
Quantified events
Showed big events
Obtained Fourier amplitudes
Assessed significance of Fourier amplitudes
Grouped spikes together
Processed spike groups
Removed spikes


In [ ]:
%%time
# Analyse minievents
reload(minievents)
MEH = minievents.MiniEventHandler(EA.data[-1],mask=EA.mask,event_number_barrier=10000,bayes_bound=0.0,score_bound=1.5)
MEH.run()

Classified background noise
8.14209008217


In [ ]:
# 19758
MEH.event_box.keys()

In [24]:
plt.loglog(MEH.event_box['rise'],MEH.event_box['decay'],ls='none',marker='o')

In [25]:
from sklearn.cluster import KMeans
X = np.stack([np.log10(MEH.event_box['rise']),np.log10(MEH.event_box['decay'])]).T
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
kmeans.labels_
for i in range(n_clusters):
    inclust = np.where(kmeans.labels_==i)
    plt.loglog(MEH.event_box['rise'][inclust],MEH.event_box['decay'][inclust],ls='none',marker='o')

/Users/douglasboubert/anaconda3/envs/py27/lib/python2.7/site-packages/matplotlib/axes/_base.py:2961: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=1e-10, right=1e-10
  'left=%s, right=%s') % (left, right))


In [ ]:
# Plot all the events together
e_time = np.linspace(0.,10.,1000)
colorlist= ['deepskyblue','orangered','forestgreen','crimson']
for i in range(MEH.event_box['N']):
    plt.plot(e_time,(MEH.event_box['scale'][i])*MEH._biexponential(e_time,RISE=MEH.event_box['rise'][i],DECAY=MEH.event_box['decay'][i]),c=colorlist[kmeans.labels_[i]],alpha=0.2)

In [26]:
plt.loglog(MEH.event_box['peak_time'],MEH.event_box['rise'],ls='none',marker='o')

In [27]:
for i in range(MEH.event_box['rchi2'].size):
    if MEH.event_box['rchi2'][i]>1.2:
        print MEH.event_box['rchi2'][i], int(MEH.event_box['t'][i]/MEH.dt), MEH.event_box['t'][i]

In [ ]:
bins = np.arange(-1,10,0.2)
plt.hist(MEH.event_box['bayes_factor'],bins)

In [28]:
MEH._post_process_event_box(significance_threshold=2.0,charge_threshold=5.0,rchi2_threshold=1.0)
plt.plot(MEH.t,MEH.data)
plt.plot(MEH.t,MEH.data_residual)
plt.plot(MEH.t,MEH.score)
plt.scatter(MEH.raw_event_box['t'],np.zeros(MEH.raw_event_box['t'].size),zorder=10,c='purple')
plt.scatter(MEH.event_box['t'],np.zeros(MEH.event_box['t'].size),zorder=10)
#plt.xlim([17355,17365])
#plt.ylim([-20,20])

In [ ]:
#MEH.event_box['rchi2'][340]
MEH.data.size

In [ ]:
bins= np.arange(0,100,1)
plt.hexbin(MEH.event_box['area'],-MEH.event_box['scale']/MEH.event_box['noise'],mincnt=1)

In [ ]:
MEH._post_process_event_box()
plt.plot(MEH.t,MEH.data)
plt.plot(MEH.t,MEH.data_residual)
plt.plot(MEH.t,MEH.score)
plt.scatter(MEH.raw_event_box['t'],np.zeros(MEH.raw_event_box['t'].size),zorder=10,c='purple')
colorlist= ['deepskyblue','orangered','forestgreen','crimson']
for ic in range(n_clusters):
    inclust = np.where(kmeans.labels_==ic)
    plt.scatter(MEH.event_box['t'][inclust],np.zeros(MEH.event_box['t'][inclust].size),zorder=10,c=colorlist[ic])

In [ ]:
_score = -np.fft.irfft(MEH.ft_data * MEH.ft_template.conjugate(),n=MEH.data.size)
#_SCORE = (_score-MEH.score_med)/MEH.score_std
_SCORE = (_score-left_score_med)/left_score_std
sc_bins = np.arange(start=_SCORE.min()-1.0,stop=_SCORE.max()+1.0,step=0.1)
sc_midbins = (sc_bins[1:]+sc_bins[:-1])/2.0
lnkde = stats.gaussian_kde(_SCORE).logpdf(sc_midbins)
sc_lnkde_interp = interpolate.interp1d(sc_midbins,lnkde)
sc_lnev = 2.0*(sc_lnkde_interp(_SCORE)-stats.norm.logpdf(_SCORE)+MEH.log_bayes_weight)

In [ ]:
bins = np.arange(-10,10,0.01)
plt.hist(sc_lnev,bins)
plt.yscale('log')

In [ ]:
from matplotlib.colors import LogNorm
plt.hexbin(sc_lnev,_SCORE,mincnt=1,norm=LogNorm(),gridsize=200,extent=[-10,10,-5,5])

In [ ]:
int(18413/MEH.dt)

In [ ]:
valid = np.where((_SCORE>1.5))
plt.plot(MEH.t,MEH.data)
plt.scatter(MEH.t[valid],np.zeros(valid[0].size),zorder=10,c='orangered')
plt.plot(MEH.t,_SCORE)
plt.plot(MEH.t,_score)

In [ ]:
plt.plot(sc_midbins,lnkde)
plt.plot(sc_midbins,stats.norm.logpdf(sc_midbins,scale=1.0))

In [ ]:
plt.hist(_SCORE,100)
plt.yscale('log')

In [ ]:
all_score_med, all_score_std = utilities.rolling(_score,FUNC='SUFFICIENT_STATISTICS',WINDOW=int(0.1*MEH.median_window))
left_score_med, left_score_std = utilities.rolling(_score,FUNC='SUFFICIENT_STATISTICS_LEFT',WINDOW=int(0.1*MEH.median_window))

In [ ]:
_LEFT_SCORE = (_score-left_score_med)/left_score_std
_ALL_SCORE = (_score-all_score_med)/all_score_std

In [ ]:
bins = np.linspace(-10,20,100)
plt.hist(_LEFT_SCORE,bins,alpha=0.5)
plt.hist(_ALL_SCORE,bins,alpha=0.5)
plt.yscale('log')

In [ ]:
# Attempt at doing two events simultaneously
from scipy import optimize
from scipy import signal
# Pull out event and play with fitting
_npoints = int(10.0/MEH.dt)
#_idx = 19758
#_idx = 83065
#_idx=86431
#_idx = 12108
#_idx = 144216
#_idx = 92527
#_idx = 52164
#_idx = int(17225.0/MEH.dt)
_idx = int(13280.0/MEH.dt)
NOISE = MEH.noise_std[_idx]
MEDIAN = MEH.noise_med[_idx]


# Are there multiple significant peaks?
loop_bool = True
_peaks = [_npoints]

_previous_peaks = []
_first_peak = _idx+min(_peaks)-_npoints
_last_peak = _idx+max(_peaks)-_npoints
_peak_gap = 2
_backwards_idx = 5
_left_extension = _npoints
_right_extension = int(1.5*_npoints)
_default_peak_time = MEH._biexponential_peak_time(RISE=MEH.default_rise,DECAY=MEH.default_decay)
while loop_bool:
    _score = MEH.score_initial[_first_peak-_left_extension:_last_peak+_right_extension]
    _data = MEH.data[_first_peak-_left_extension:_last_peak+_right_extension]
    _time = MEH.t[_first_peak-_left_extension:_last_peak+_right_extension]
    _peak_gap = int(1.0/MEH.dt)
    _peaks = [] # we start with the original peak
    _peaks_heights = []
    _suggested_peaks, _peaks_props = signal.find_peaks(-_data,prominence=NOISE*2,height = 5.0,width=2)
    #_suggested_peaks = signal.find_peaks_cwt(_score, np.arange(5,10))
    print(_suggested_peaks)
    for _peak_idx,_peak_height in zip(_suggested_peaks,_peaks_props['peak_heights']):
        _mad_idx = int((_time[_peak_idx]-_default_peak_time)/MEH.dt)
        _max_score = MEH.score_initial[_mad_idx]
        if _max_score>0.5:
            _peaks.append(_peak_idx)
            _peaks_heights.append(-_peak_height)
    print _peaks
    
    if _peaks == _previous_peaks:
        loop_bool=False
    _n_peaks = len(_peaks)
    print len(_peaks), _n_peaks
    _first_peak = _first_peak+min(_peaks)-_npoints
    _last_peak = _first_peak+max(_peaks)-_npoints
    _previous_peaks = _peaks

# Pull out time and data information
#_first_peak = _idx+min(_peaks)-_npoints
#_last_peak = _idx+max(_peaks)-_npoints
_time = MEH.t[_first_peak-_left_extension:_last_peak+_right_extension]
_data = MEH.data[_first_peak-_left_extension:_last_peak+_right_extension]
_score = MEH.score_initial[_first_peak-_left_extension:_last_peak+_right_extension]
_PEAKS = [_p for _p in _peaks]

# Gather useful quantities
_data_scale = np.abs(_data.max()-_data.min())
_range_start_t = [3.*MEH.default_rise,1.*MEH.default_rise]

_weights= np.exp(-((_time[:,np.newaxis]-_time[_peaks])**2.0/(2.0*(3.0*MEH.default_decay)**2.0))).sum(axis=1)

def _mod_template(T,X):
    SCALE, RISE, DECAY, START_T = X
    return SCALE * MEH._biexponential(T-START_T,RISE=RISE,DECAY=DECAY)

def _super_model(T,X,_NMODEL):
    OFFSET, GRADIENT, EVENT_PARAMS = X[0], X[1], X[2:].reshape((_NMODEL,4))
    _model = OFFSET + GRADIENT*(T-T[_npoints])
    for _model_index in range(_NMODEL):
        _model += _mod_template(T,EVENT_PARAMS[_model_index])
    return _model

def _sub_model(T,X,_NMODEL,_nMODEL):
    OFFSET, GRADIENT, EVENT_PARAMS = X[0], X[1], X[2:].reshape((_NMODEL,4))
    _model = OFFSET + GRADIENT*(T-T[_npoints]) + _mod_template(T,EVENT_PARAMS[_nMODEL])
    return _model

_gauss_norm = -0.5*np.log(2.0*np.pi*NOISE**2.0)
def _target(X,_NMODEL=1):
    _model = _super_model(_time,X,_NMODEL)
    return (-(-0.5*(_data-_model)**2.0/NOISE**2.0+_gauss_norm)*_weights).sum()+1.5*np.log(1.0+X[1]**2.0)

# Set up initial location and bounds
_X0 = [MEDIAN,0.0]
#_delta = abs((_data[-1]-_data[0])/(_time[-1]-_time[0]))
_bnds = [(_data.min(), _data.max()),(-1.0,1.0)]
for _p,_h in zip(_PEAKS,_peaks_heights):
    _mean_start_t = _time[_p]-_default_peak_time-MEH.default_rise
    _X0 += [_h, MEH.default_rise, MEH.default_decay,_mean_start_t]
    print (_h*0.5, _h*1.5)
    _bnds += [(_h*1.5, _h*0.5), (1e-10, 1e5), (1e-2, 1e2), (_mean_start_t-_range_start_t[0], _mean_start_t+_range_start_t[1])]
_opt = {'gtol':1e-10,'ftol':1e-10,'maxfun':250000}
_res = optimize.minimize(_target,_X0,method='L-BFGS-B', tol=1e-10, bounds=_bnds, options=_opt, args=(len(_peaks)))
RCHI2 = np.mean(np.abs(_data-_super_model(_time,_res['x'],len(_peaks)))/NOISE)
#OFFSET, SCALE, RISE, DECAY, START_T = _res['x']

In [ ]:
_res

In [ ]:
plt.plot(_time,_data)
plt.plot(_time,_score)
plt.plot(_time,_super_model(_time,_res['x'],len(_peaks)))
#plt.plot(_time,_data-_super_model(_time,_res['x'],len(_peaks)))
plt.scatter(_time[_peaks],np.zeros(len(_peaks)),zorder=10,c='orangered')
#plt.scatter(_res['x'][5::4],np.ones(len(_peaks)),zorder=10,c='purple')

In [ ]:
from scipy import signal
plt.plot(_time,_data)
plt.plot(_time,_score)
tmppeaks,props = signal.find_peaks(-_data,prominence=NOISE*2,height = 5.0,width=2)
truepeaks = np.array([i for i in tmppeaks if _score[i-5:i].max()>0.5])
plt.scatter(_time[tmppeaks],np.zeros(tmppeaks.size),zorder=10,c='purple')
plt.scatter(_time[truepeaks],np.zeros(truepeaks.size),zorder=10,c='orangered')

In [ ]:
plt.plot(_time,_weights)

In [ ]:
int(17200/MEH.dt)

In [ ]:

plt.plot(MEH.t[_idx-_npoints:_idx+_npoints*2],MEH.data[_idx-_npoints:_idx+_npoints*2])
plt.plot(MEH.t[_idx-_npoints:_idx+_npoints*2],MEH.data_residual[_idx-_npoints:_idx+_npoints*2])
plt.plot(MEH.t[_idx-_npoints:_idx+_npoints*2],MEH.score[_idx-_npoints:_idx+_npoints*2])

#plt.scatter(MEH.t[_idx-_npoints:_idx+_npoints*2][highpeaks],np.zeros(highpeaks.size))

In [ ]:
from scipy import signal
plt.plot(MEH.t,MEH.data)
plt.plot(MEH.t,MEH.score)
tmppeaks,props = signal.find_peaks(-MEH.data,prominence=MEH.noise_std*2,height = 5.0,width=2,wlen=20)
truepeaks = np.array([i for i in tmppeaks if MEH.score[i-5:i].max()>3.0])
plt.scatter(MEH.t[truepeaks],np.zeros(truepeaks.size),zorder=10,c='orangered')

In [ ]:
int(16530.0/MEH.dt)

In [ ]:
data = EA.data[-1]
import pandas as pd
def rolling(DATA,WINDOW=50,FUNC='MEDIAN',EDGE_METHOD='SHIFTING'):
    def cleaning(RAW_ROLLING):
        RAW_ROLLING[:WINDOW] = np.roll(RAW_ROLLING,-WINDOW)[:WINDOW]
        RAW_ROLLING[-WINDOW:] = np.roll(RAW_ROLLING,WINDOW)[-WINDOW:]
        return RAW_ROLLING
    
    if FUNC=='MEDIAN':
        raw_rolling = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).median().values.flatten()
    elif FUNC=='MEAN':
        raw_rolling = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).mean().values.flatten()
    elif FUNC=='STD':
        raw_rolling = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).std().values.flatten()
    elif FUNC=='SUM':
        raw_rolling = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).sum().values.flatten()
    elif FUNC=='SMOOTH_MEDIAN':
        raw_rolling_q1 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.40).values.flatten()
        raw_rolling_q3 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.60).values.flatten()
        raw_rolling = np.median(np.stack([raw_rolling_q1,raw_rolling_q3]),axis=0)
    elif FUNC=='SUFFICIENT_STATISTICS':
        raw_rolling_q1 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.25).values.flatten()
        raw_rolling_q2 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.50).values.flatten()
        raw_rolling_q3 = pd.DataFrame(DATA).rolling(window=WINDOW,center=True).quantile(0.75).values.flatten()
        raw_rolling_std = (raw_rolling_q3-raw_rolling_q1)/1.349
        return cleaning(raw_rolling_q2),cleaning(raw_rolling_std)
        
    return cleaning(raw_rolling)
        

In [ ]:
med, std =rolling(data,FUNC='SUFFICIENT_STATISTICS',WINDOW=MEH.median_window)

In [ ]:
def _triexponential(T,RISE1=1.0,RISE2=2.0,DECAY=3.0,F=0.5):
    NORM = 1.0#1/((DECAY/(RISE+DECAY))*(RISE/(RISE+DECAY))**(RISE/DECAY))
    retarray = np.zeros(T.shape[0])
    postime = T>-10.
    retarray[postime] = NORM*(1.0-F*np.exp(-T[postime]/RISE1)-(1.0-F)*np.exp(-T[postime]/RISE2))*(np.exp(-T[postime]/DECAY))
    return retarray#/retarray.max()

rise1 = 2.0
rise2 = 10.0
decay = 5.0
T = np.linspace(0.,10.,10000)
#frange = np.linspace(-rise1/(rise2-rise1),1.0,100)
frange = np.linspace(0.0,1.0,100)
for f in frange:
    plt.plot(T,_triexponential(T,RISE1=rise1,RISE2=rise2,DECAY=decay,F=f),c=plt.cm.viridis((f-frange.min())/(frange.max()-frange.min())))

In [ ]:
max_x = scipy.optimize.fmin(lambda x: -f(x), 0)

In [ ]:
def _triexponential(T,RISE1=1.0,RISE2=2.0,DECAY=3.0,F=0.5):
    NORM = 1.0#1/((DECAY/(RISE+DECAY))*(RISE/(RISE+DECAY))**(RISE/DECAY))
    retarray = np.zeros(T.shape[0])
    retarray = NORM*(1.0-F*np.exp(-T/RISE1)-(1.0-F)*np.exp(-T/RISE2))*(np.exp(-T/DECAY))
    return retarray#/retarray.max()

rise1 = 1.0
rise2 = 2.0
decay = 5.0
f = 0.5
T = np.linspace(0.,10.,10000)
plt.plot(T,_triexponential(T,RISE1=rise1,RISE2=rise2,DECAY=decay,F=f))

In [ ]:
# Use minimize_scalar
from math import exp
rise1 = 1.0
rise2 = 2.0
decay = 3.0
f = 0.5
T = np.linspace(0.,10.,10000)
def _triexp_scalar(x):
    return -(1.0-f*exp(-x/rise1)-(1.0-f)*exp(-x/rise2))*exp(-x/decay)
_res = optimize.minimize_scalar(_triexp_scalar)

In [ ]:
from math import exp
rise1 = 1.0
rise2 = 2.0
decay = 3.0
f = 0.5
T = np.linspace(0.,10.,10000)
def _triexp_scalar(x):
    return -(1.0-f*exp(-x/rise1)-(1.0-f)*exp(-x/rise2))*exp(-x/decay)
_res = optimize.minimize_scalar(_triexp_scalar,method='golden',tol=1e-3)

In [ ]:
_res

In [ ]:
%timeit optimize.minimize_scalar(_triexp_scalar,method='brent')

In [ ]:
%timeit optimize.minimize_scalar(_triexp_scalar,method='golden',tol=1e-3)

In [ ]:
%timeit optimize.minimize_scalar(_triexp_scalar,method='bounded',bounds=(0,10))

In [ ]:
from math import exp
def _triexponential_peak(RISE,ETA,DECAY,F):
    RISE1 = RISE
    RISE2 = RISE*ETA
    G0_A = F*(1.0+DECAY/RISE1)
    G0_B = (1.0-F)*(1.0+DECAY/RISE2)
    G1_A = -G0_A/RISE1
    G1_B = -G0_B/RISE2
    G2_A = -G1_A/RISE1
    G2_B = -G1_B/RISE2
    def _G0(x):
        return -1.0+G0_A*exp(-x/RISE1)+G0_B*exp(-x/RISE2)
    def _G1(x):
        return G1_A*exp(-x/RISE1)+G1_B*exp(-x/RISE2)
    def _G2(x):
        return G2_A*exp(-x/RISE1)+G2_B*exp(-x/RISE2)

    root = optimize.newton(_G0, 1.0, fprime=_G1,fprime2=_G2)
    return root

def _triexponential(T,RISE=1.0,ETA=1.0,DECAY=1.0,F=0.5,NORMED=True):
    RISE1 = RISE
    RISE2 = RISE*ETA
    retarray = np.zeros(T.shape[0])
    retarray = (1.0-F*np.exp(-T/RISE1)-(1.0-F)*np.exp(-T/RISE2))*(np.exp(-T/DECAY))
    if NORMED == True:
        PEAK = _triexponential_peak(RISE,ETA,DECAY,F)
        NORM = _triexponential(np.array([PEAK]),RISE,ETA,DECAY,F,NORMED=False)[0]
        retarray /= NORM
    return retarray

def _triexponential_area(RISE=1.0,ETA=1.0,DECAY=1.0,F=0.5,NORMED=True):
    RISE1 = RISE
    RISE2 = RISE*ETA
    AREA = ((DECAY+(1.0-F)*RISE1+F*RISE2)/((1.0+RISE1/DECAY)*(1.0+RISE2/DECAY)))
    if NORMED == True:
        PEAK = _triexponential_peak(RISE,ETA,DECAY,F)
        NORM = _triexponential(np.array([PEAK]),RISE,ETA,DECAY,F,NORMED=False)[0]
        AREA /= NORM
    return AREA

In [ ]:
rise = 1.0
eta = 2.0
decay = 5.0
f = 0.5
T = np.linspace(0.,10.,10000)
#varrange = np.logspace(0.0,5,100)
varrange = np.linspace(0.,1.0,100)

for v in varrange:
    plt.plot(T,_triexponential(T,RISE=rise,ETA=eta,DECAY=decay,F=v),c=plt.cm.viridis((v)/(1.0)))
plt.plot(T,_triexponential(T,RISE=rise,ETA=eta,DECAY=decay,F=0.5),c='r')

In [ ]:
rise = 1.0
eta = 2.0
decay = 5.0
f = 0.5
T = np.linspace(0.,10.,10000)
varrange = np.logspace(0.0,10,100)

for v in varrange:
    plt.plot(T,_triexponential(T,RISE=rise,ETA=v,DECAY=decay,F=f),c=plt.cm.viridis(np.log10(v)/(5.0)))
plt.plot(T,_triexponential(T,RISE=rise,ETA=eta,DECAY=decay,F=0.5),c='r')

In [ ]:
np.log10(varrange)

In [ ]:
def integrand(t):
    return _triexponential(np.array([t]),RISE=1.0,ETA=2.0,DECAY=5.0,F=0.8,NORMED=True)[0]
integrate.quad(integrand,0,np.inf)

In [ ]:
_triexponential_area(RISE=1.0,ETA=2.0,DECAY=5.0,F=0.8)

In [ ]:
%timeit _triexponential_peak(RISE1=1.0,RISE2=2.0,DECAY=5.0,F=0.5)

In [ ]:
%timeit optimize.newton(_G0, 0.0)

In [ ]:
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import spsolve

def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

In [ ]:
plt.plot(MEH.t,EA.data[-1])
plt.plot(MEH.t,baseline_als(EA.data[-1],10000.,0.001))

In [ ]:
def highpass_filter(y, sr):
    filter_stop_freq = 10  # Hz
    filter_pass_freq = 20  # Hz
    filter_order = 1001

    # High-pass filter
    nyquist_rate = sr / 2.
    desired = (0, 0, 1, 1)
    bands = (0, filter_stop_freq, filter_pass_freq, nyquist_rate)
    filter_coefs = signal.firls(filter_order, bands, desired, nyq=nyquist_rate)

    # Apply high-pass filter
    filtered_audio = signal.filtfilt(filter_coefs, [1], y)
    return filtered_audio

In [ ]:
plt.plot(MEH.t,EA.data[-1])
plt.plot(MEH.t,highpass_filter(EA.data[-1],1e3/MEH.dt))

In [ ]:
MEH.dt